In [1]:
import mlflow
import pandas as pd
from pathlib import Path
from hydra import compose, initialize

In [2]:
with initialize(config_path="../configs/filepaths", version_base=None):
    fps = compose(config_name="base")

def get_best_epochs(run_id: str) -> int:
    dir = Path(fps.tracking_uri.removeprefix("file:///")) / "0" / run_id / "checkpoints"
    ckpts = list(dir.glob('*.ckpt'))
    val_epochs = [
        (float(ckpt.stem.split('-')[4]), int(ckpt.stem.split('-')[2]))
        for ckpt in ckpts
    ]
    val_epochs = sorted(val_epochs, key=lambda x: x[0], reverse=True)
    if len(val_epochs) == 0:
        return -1
    return val_epochs[0][1] + 1  # epochs are 0-indexed in filenames

In [3]:
mlflow.set_tracking_uri(fps.tracking_uri)

In [4]:
filter_split = "params.'data/split_idx' = '0'"

runs = mlflow.search_runs(
    filter_string=filter_split,
)

In [5]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.val_loss,metrics.val/accuracy,metrics.epoch,metrics.val/roc,...,params.warmup_epochs,params.X_d_transform,params.batch_norm,params.init_lr,params.metrics,params.model/radius,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName
0,cfb7fdbeb07a4967affe7f4485957236,0,RUNNING,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 09:54:59.472000+00:00,NaT,0.681740,0.912826,20.0,0.937603,...,None,None,None,None,None,None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,bouncy-lark-884
1,8a016e73833f4a9f8e3127ab05a8f448,0,RUNNING,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:27:55.540000+00:00,NaT,0.497429,0.855373,16.0,0.867821,...,None,None,None,None,None,None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,resilient-fish-941
2,e3738fe315f64c928c44fe4f8ac6de24,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:27:55.540000+00:00,2025-12-15 12:29:26.686000+00:00,0.802645,0.956274,24.0,0.972450,...,2,None,True,0.0001,"[BinaryF1Metric(task_weights=[[1.0]]), BinaryP...",None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,unleashed-colt-871
3,ec68f688ce774c09a5811c38a38b1f8f,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:27:35.721000+00:00,2025-12-15 09:40:41.798000+00:00,0.640662,0.890778,24.0,0.920642,...,None,None,None,None,None,2,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,dapper-calf-261
4,0c19be31db744a0e8a0c87905f46c81a,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:17:09.981000+00:00,2025-12-15 11:10:55.009000+00:00,0.455349,0.826837,24.0,0.837735,...,None,None,None,None,None,2,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,big-croc-84
5,6c60b4f3d0d74c4a9c71ab58d8ca79df,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:16:53.174000+00:00,2025-12-15 13:26:15.782000+00:00,0.607795,0.911444,24.0,0.910619,...,2,None,True,0.0001,"[BinaryF1Metric(task_weights=[[1.0]]), BinaryP...",None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,grandiose-hen-412
6,825c6f409237497991112112d2432f48,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 03:54:04.160000+00:00,2025-12-15 11:15:43.168000+00:00,0.655347,0.943353,24.0,0.947453,...,2,None,True,0.0001,"[BinaryF1Metric(task_weights=[[1.0]]), BinaryP...",None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,rebellious-finch-358
7,d651d3368fc8427383ab68266359966f,0,RUNNING,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 03:54:03.281000+00:00,NaT,0.388875,0.859467,17.0,0.784445,...,None,None,None,None,None,None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,calm-sponge-3
8,707754f2458b467d871b76e7cf93b8df,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 03:53:38.219000+00:00,2025-12-15 10:33:58.676000+00:00,0.450442,0.882118,24.0,0.800193,...,None,None,None,None,None,2,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,industrious-loon-772
9,4b7d87b9ffba476e82cadf81c28dd3f8,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-14 22:45:07.456000+00:00,2025-12-15 02:25:48.744000+00:00,0.526610,0.879614,24.0,0.877561,...,2,None,True,0.0001,"[BinaryF1Metric(task_weights=[[1.0]]), BinaryP...",None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,silent-croc-665


In [6]:

runs['best_epoch'] = runs['run_id'].apply(get_best_epochs)
runs[['params.model/name', 'params.data/split_strategy', 'params.data/negative_sampling', 'best_epoch', 'run_id',]].sort_values(
    by=['params.model/name', 'params.data/split_strategy', 'params.data/negative_sampling']
)

,params.model/name,params.data/split_strategy,params.data/negative_sampling,best_epoch,run_id
9,bom,random_reaction,alternate_reaction_center,24,4b7d87b9ffba476e82cadf81c28dd3f8
14,bom,random_reaction_center,alternate_reaction_center,24,8cff9872dc0441e5a7d1561d75e41149
15,bom,rcmcs,random,9,7725706057394618bc1c2fad58614627
2,cgr,random_reaction,alternate_reaction_center,15,e3738fe315f64c928c44fe4f8ac6de24
5,cgr,random_reaction_center,alternate_reaction_center,24,6c60b4f3d0d74c4a9c71ab58d8ca79df
6,cgr,rcmcs,random,12,825c6f409237497991112112d2432f48
0,drfp,random_reaction,alternate_reaction_center,21,cfb7fdbeb07a4967affe7f4485957236
1,drfp,random_reaction_center,alternate_reaction_center,9,8a016e73833f4a9f8e3127ab05a8f448
7,drfp,rcmcs,random,15,d651d3368fc8427383ab68266359966f
3,mfp,random_reaction,alternate_reaction_center,24,ec68f688ce774c09a5811c38a38b1f8f


In [7]:
filter_split = "params.'data/split_idx' = '-1'"

test_runs = mlflow.search_runs(
    filter_string=filter_split,
)

test_runs[['params.model/name', 'params.data/split_strategy', 'params.data/negative_sampling', 'metrics.val/roc', 'run_id']].sort_values(
    by=['params.model/name', 'params.data/split_strategy', 'params.data/negative_sampling']
)

,params.model/name,params.data/split_strategy,params.data/negative_sampling,metrics.val/roc,run_id
6,bom,random_reaction,alternate_reaction_center,0.941901,55c062d229394c4f82828814300c0f2b
10,bom,random_reaction_center,alternate_reaction_center,0.729101,35d782fa639e4035b08d5c74a4f002f9
14,bom,rcmcs,random,0.908215,d1a3cc3f1d36488e8222efda0d595914
0,cgr,random_reaction,alternate_reaction_center,0.982500,120f090f4b3e471a965979401b3271ae
1,cgr,random_reaction_center,alternate_reaction_center,0.807885,34cda1e3f64a4b18a98894ed0e86be86
2,cgr,rcmcs,random,0.914909,e627a08cec3c47e18b4fd9f1ade6b9fe
3,mfp,random_reaction,alternate_reaction_center,0.852761,7e836b6654ac4b9aba81fe692b059c5d
4,mfp,random_reaction_center,alternate_reaction_center,0.679670,ec2310e5872f47de8b9c81d1edce26b9
5,mfp,rcmcs,random,0.755406,04cc886dade64a68a84b151ddd92cef2
8,rc_agg,random_reaction,alternate_reaction_center,0.985613,1d6c62f5f2d44b3fa82851639fb44ead


In [8]:
filter_split = "params.'data/split_idx' = '-2'"

prod_runs = mlflow.search_runs(
    filter_string=filter_split,
)

In [9]:
prod_runs[['params.model/name', 'params.data/split_strategy', 'params.data/negative_sampling', 'metrics.epoch', 'run_id',]].sort_values(
    by=['params.model/name', 'params.data/split_strategy', 'params.data/negative_sampling']
)

,params.model/name,params.data/split_strategy,params.data/negative_sampling,metrics.epoch,run_id
6,bom,random_reaction_center,alternate_reaction_center,23.0,fafcf359d8864d1a8d012d65eb4f350d
12,bom,rcmcs,random,8.0,2fcb812a06c841d282d378d9f99e64c9
0,cgr,random_reaction_center,alternate_reaction_center,14.0,25964e2b2fcb4ee1aa6f5487866209bd
1,cgr,rcmcs,random,11.0,4c7872263bba4bd2952057c3146c7c9f
2,mfp,random_reaction_center,alternate_reaction_center,23.0,b736cf547f5b46a0b8eb0308bddcabbd
3,mfp,rcmcs,random,11.0,baa4374b85ca4deaa6df54cdc14617b7
4,rc_agg,random_reaction_center,alternate_reaction_center,5.0,9226ba678bcd42fbb95b44979deeabff
13,rc_agg,rcmcs,random,11.0,1fbaa827a8384e8da2fadaaa945fce13
5,rc_cxn,random_reaction_center,alternate_reaction_center,11.0,99fb3dad48464fd8a2af365b5fd57621
14,rc_cxn,rcmcs,random,8.0,722de30540f6401d8d7fd12e8c2d0b4a


In [10]:
for _, row in runs.iterrows():
    print(row['run_id'])

for _, row in test_runs.iterrows():
    print(row['run_id'])

cfb7fdbeb07a4967affe7f4485957236
8a016e73833f4a9f8e3127ab05a8f448
e3738fe315f64c928c44fe4f8ac6de24
ec68f688ce774c09a5811c38a38b1f8f
0c19be31db744a0e8a0c87905f46c81a
6c60b4f3d0d74c4a9c71ab58d8ca79df
825c6f409237497991112112d2432f48
d651d3368fc8427383ab68266359966f
707754f2458b467d871b76e7cf93b8df
4b7d87b9ffba476e82cadf81c28dd3f8
ac3565b3fe984552922766a57944a54a
103f73d69ff640c99bb8645a2e9146c6
9f917717289f4e9a89564564a38361f3
28abcb5c5573447fa20f212703d5d597
8cff9872dc0441e5a7d1561d75e41149
7725706057394618bc1c2fad58614627
0917682b20b545a59ed98c2d423c56d1
e109a0a7d3564f99bb6688796405dbf1
6bb1753ed0b14a6fb6eddb0d0b3679a5
e54a25eafc65463a95dee11b4a46e4cd
f0d279b0ffed4544bc23a81327e449d0
120f090f4b3e471a965979401b3271ae
34cda1e3f64a4b18a98894ed0e86be86
e627a08cec3c47e18b4fd9f1ade6b9fe
7e836b6654ac4b9aba81fe692b059c5d
ec2310e5872f47de8b9c81d1edce26b9
04cc886dade64a68a84b151ddd92cef2
55c062d229394c4f82828814300c0f2b
56c0235555af4e4c98dd6293c824cc2f
1d6c62f5f2d44b3fa82851639fb44ead
30ed98a93b

In [11]:
join_cols = ['params.model/name', 'params.data/split_strategy', 'params.data/negative_sampling']
in_out = pd.merge(runs, test_runs, how='inner', on=join_cols, suffixes=('_in', '_out'))
in_out.head()

,run_id_in,experiment_id_in,status_in,artifact_uri_in,start_time_in,end_time_in,metrics.val_loss_in,metrics.val/accuracy_in,metrics.epoch_in,metrics.val/roc_in,...,params.data/seed_out,params.batch_norm_out,params.data/n_splits_out,params.metrics_out,params.model/vec_len_out,params.model/radius_out,tags.mlflow.user_out,tags.mlflow.source.name_out,tags.mlflow.source.type_out,tags.mlflow.runName_out
0,e3738fe315f64c928c44fe4f8ac6de24,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:27:55.540000+00:00,2025-12-15 12:29:26.686000+00:00,0.802645,0.956274,24.0,0.972450,...,1234,True,3,"[BinaryF1Metric(task_weights=[[1.0]]), BinaryP...",None,None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,incongruous-ram-908
1,ec68f688ce774c09a5811c38a38b1f8f,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:27:35.721000+00:00,2025-12-15 09:40:41.798000+00:00,0.640662,0.890778,24.0,0.920642,...,1234,None,3,None,2048,2,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,wistful-horse-72
2,0c19be31db744a0e8a0c87905f46c81a,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:17:09.981000+00:00,2025-12-15 11:10:55.009000+00:00,0.455349,0.826837,24.0,0.837735,...,1234,None,3,None,2048,2,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,welcoming-shad-999
3,6c60b4f3d0d74c4a9c71ab58d8ca79df,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 04:16:53.174000+00:00,2025-12-15 13:26:15.782000+00:00,0.607795,0.911444,24.0,0.910619,...,1234,True,3,"[BinaryF1Metric(task_weights=[[1.0]]), BinaryP...",None,None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,fearless-hare-64
4,825c6f409237497991112112d2432f48,0,FINISHED,file:///projects/p30041/spn1560/hiec/results/r...,2025-12-15 03:54:04.160000+00:00,2025-12-15 11:15:43.168000+00:00,0.655347,0.943353,24.0,0.947453,...,1234,True,3,"[BinaryF1Metric(task_weights=[[1.0]]), BinaryP...",None,None,spn1560,/home/spn1560/hiec/scripts/train.py,LOCAL,caring-fox-87


In [12]:
for _, row in in_out.iterrows():
    print(f"{row['run_id_in']}: {row['run_id_out']}")

e3738fe315f64c928c44fe4f8ac6de24: 120f090f4b3e471a965979401b3271ae
ec68f688ce774c09a5811c38a38b1f8f: 7e836b6654ac4b9aba81fe692b059c5d
0c19be31db744a0e8a0c87905f46c81a: ec2310e5872f47de8b9c81d1edce26b9
6c60b4f3d0d74c4a9c71ab58d8ca79df: 34cda1e3f64a4b18a98894ed0e86be86
825c6f409237497991112112d2432f48: e627a08cec3c47e18b4fd9f1ade6b9fe
707754f2458b467d871b76e7cf93b8df: 04cc886dade64a68a84b151ddd92cef2
4b7d87b9ffba476e82cadf81c28dd3f8: 55c062d229394c4f82828814300c0f2b
ac3565b3fe984552922766a57944a54a: 56c0235555af4e4c98dd6293c824cc2f
103f73d69ff640c99bb8645a2e9146c6: 30ed98a93bb149f6b7b6c08779e95915
9f917717289f4e9a89564564a38361f3: 874ef317e1b7403c9be3665c3c301d90
28abcb5c5573447fa20f212703d5d597: 1d6c62f5f2d44b3fa82851639fb44ead
8cff9872dc0441e5a7d1561d75e41149: 35d782fa639e4035b08d5c74a4f002f9
7725706057394618bc1c2fad58614627: d1a3cc3f1d36488e8222efda0d595914
0917682b20b545a59ed98c2d423c56d1: af29e2f62cf34bf2824dc8c72e086da5
e109a0a7d3564f99bb6688796405dbf1: da9ed201898a451a8a822cf1a30e